In [14]:
from PIL import Image
import numpy as np
import cv2
import os
import os, json, cv2, random

from detectron2.data.datasets import register_coco_instances
from detectron2.data import MetadataCatalog, DatasetCatalog
from detectron2.utils.visualizer import Visualizer
from detectron2.engine import DefaultPredictor
from detectron2.config import get_cfg
from detectron2 import model_zoo



cfg = get_cfg()
cfg.merge_from_file(model_zoo.get_config_file("COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_3x.yaml"))
cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.7   # set a custom testing threshold
cfg.MODEL.ROI_HEADS.NUM_CLASSES = 2
# cfg.MODEL.WEIGHTS = "./OutputModels/model_final.pth"
cfg.MODEL.WEIGHTS = "./OutputModelsSegmentation/model_final.pth"
cfg.MODEL.DEVICE = "cpu"

predictor = DefaultPredictor(cfg)


upload_dir = "./SegOut/"
predict_dir = "./SegImages/"
os.makedirs(upload_dir,exist_ok=True)
os.makedirs(predict_dir,exist_ok=True)

In [3]:
register_coco_instances("val",{},"Data/val.json","Data/val")

val_metadata = MetadataCatalog.get('val')
val_dicts = DatasetCatalog.get('val')


Category ids in annotations are not in [1, #categories]! We'll apply a mapping for you.



In [4]:
val_dicts[0]

{'file_name': 'Data/val\\ISIC_0238871.jpg',
 'height': 300,
 'width': 300,
 'image_id': 0,
 'annotations': [{'iscrowd': 0,
   'bbox': [108.0, 132.0, 43.0, 61.0],
   'category_id': 0,
   'segmentation': [[108.79010494752623,
     141.22938530734632,
     114.63718140929535,
     168.9655172413793,
     129.02998500749626,
     185.60719640179911,
     146.12143928035982,
     193.7031484257871,
     151.51874062968517,
     179.46026986506746,
     144.3223388305847,
     166.4167916041979,
     132.1784107946027,
     147.07646176911544,
     126.78110944527737,
     132.68365817091455,
     116.43628185907048,
     133.43328335832084]],
   'bbox_mode': <BoxMode.XYWH_ABS: 1>}]}

In [15]:
img = Image.open(val_dicts[0]["file_name"])
imgarray = np.array(img)[:, :, ::-1]
res = predictor(imgarray)
print(res)
filename = val_dicts[0]["file_name"].split("\\")[-1:][0]
predict_fle = os.path.join(predict_dir,filename)
v = Visualizer(img, metadata=val_metadata, scale=0.8)
v = v.draw_instance_predictions(res["instances"].to("cpu"))
cv2.imwrite(f"{upload_dir}{filename}",v.get_image()[:, :, ::-1]) #save predicted images

{'instances': Instances(num_instances=1, image_height=300, image_width=300, fields=[pred_boxes: Boxes(tensor([[107.7420, 131.6396, 151.3090, 197.4466]])), scores: tensor([0.9836]), pred_classes: tensor([0]), pred_masks: tensor([[[False, False, False,  ..., False, False, False],
         [False, False, False,  ..., False, False, False],
         [False, False, False,  ..., False, False, False],
         ...,
         [False, False, False,  ..., False, False, False],
         [False, False, False,  ..., False, False, False],
         [False, False, False,  ..., False, False, False]]])])}


True

In [11]:
def CalculateArea(mask_tensor):
  # Move tensor to CPU if necessary
  if mask_tensor.device.type != "cpu":
      mask_tensor = mask_tensor.cpu()

  # Convert mask tensor to numpy array
  mask_array = mask_tensor.numpy()

  # Compute pixel size (assuming square pixels)
  # image_width = outputs["instances"].image_width
  # image_height = outputs["instances"].image_height

  image_height = 300
  image_width = 300
  pixel_size = (image_height*image_width) / np.prod(mask_array.shape[1:])
  # Compute area of each mask
  mask_areas = []
  for mask in mask_array:
      num_pixels = np.count_nonzero(mask)
      mask_area = num_pixels * pixel_size
      mask_areas.append(mask_area)

  return mask_areas

In [19]:
def ReturnMetaDeta(res):
    pred_classes = res["instances"].to("cpu").pred_classes.tolist()
    pred_scores = res["instances"].to("cpu").scores.tolist()
    pred_boxes = res["instances"].to("cpu").pred_boxes
    pred_masks = res["instances"].to("cpu").pred_masks.tolist()

    image_area = CalculateArea(res["instances"].to("cpu").pred_masks)

      # Convert the predicted class labels and scores to a dictionary
    output = {
          "Predicted": pred_classes,
          "scores": pred_scores,
          "area":image_area,
          "pred_masks":pred_masks

      }
    return output

In [22]:
for ind in np.random.randint(0,len(val_dicts),6):
    img = Image.open(val_dicts[ind]["file_name"])
    imgarray = np.array(img)[:, :, ::-1]
    res = predictor(imgarray)

    filename = val_dicts[ind]["file_name"].split("\\")[-1:][0]
    v = Visualizer(img, metadata=val_metadata, scale=0.8)
    v = v.draw_instance_predictions(res["instances"].to("cpu"))
    cv2.imwrite(f"{upload_dir}{filename}",v.get_image()[:, :, ::-1])